**Импорт библиотек**

In [25]:
!pip install transformers datasets torch scikit-learn

In [ ]:
!pip install evaluate

In [ ]:
!pip install seqeval

In [26]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
from datasets import Dataset as HFDataset
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
import evaluate
import os
os.environ["WANDB_DISABLED"] = "true"

In [27]:
file_path = "/processed_data.csv"
df = pd.read_csv(file_path)

In [28]:
model_name = "cointegrated/rubert-tiny2"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
all_labels = set()
for entities in df["entities"]:
    for entity in eval(entities):
        all_labels.add(entity["label"])

label_list = ["O"] + sorted(all_labels)
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

print("Обнаруженные метки:", label_list)

Обнаруженные метки: ['O', 'B-DATE', 'B-LAW', 'B-PER']


In [29]:
def tokenize_and_align_labels(example):
    tokens = tokenizer(example["text"], padding="max_length", truncation=True, max_length=128, return_offsets_mapping=True)
    offset_mapping = tokens.pop("offset_mapping")

    labels = ["O"] * len(tokens["input_ids"])  # Начинаем с "O" для каждого токена

    for entity in eval(example["entities"]):  # Разбираем JSON из CSV
        start, end, entity_label = entity["start"], entity["end"], entity["label"]
        for idx, (token_start, token_end) in enumerate(offset_mapping):
            if token_start >= start and token_end <= end:
                labels[idx] = entity_label  # Назначаем метку

    tokens["labels"] = [label2id[label] for label in labels]  # Преобразуем в индексы
    return tokens

In [30]:
train_texts, val_texts = train_test_split(df, test_size=0.2)

train_dataset = HFDataset.from_pandas(train_texts)
val_dataset = HFDataset.from_pandas(val_texts)

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=False)
val_dataset = val_dataset.map(tokenize_and_align_labels, batched=False)


Map:   0%|          | 0/776 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

In [31]:
model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [48]:
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id2label[label] for label in label_seq if label != -100] for label_seq in labels]
    true_predictions = [[id2label[pred] for pred, label in zip(pred_seq, label_seq) if label != -100] for pred_seq, label_seq in zip(predictions, labels)]

    return metric.compute(predictions=true_predictions, references=true_labels)

In [50]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [51]:
trainer.train()

Epoch,Training Loss,Validation Loss,Date,Law,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.017100,0.014619,"{'precision': 0.9545454545454546, 'recall': 0.9897540983606558, 'f1': 0.971830985915493, 'number': 488}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 26}",0.954545,0.928846,0.941520,0.997596
2,0.011100,0.013582,"{'precision': 0.9777327935222672, 'recall': 0.9897540983606558, 'f1': 0.9837067209775967, 'number': 488}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 26}",0.977733,0.928846,0.952663,0.998077
3,0.009800,0.013317,"{'precision': 0.9797160243407708, 'recall': 0.9897540983606558, 'f1': 0.9847094801223242, 'number': 488}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 26}",0.979716,0.928846,0.953603,0.998117


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.9545454545454546, 'recall': 0.9897540983606558, 'f1': 0.971830985915493, 'number': 488}" of type <class 'dict'> for key "eval/DATE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}" of type <class 'dict'> for key "eval/LAW" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 26}" of type <class 'dict'> for key "eval/PER" as a scalar

TrainOutput(global_step=291, training_loss=0.010237421264353487, metrics={'train_runtime': 420.4114, 'train_samples_per_second': 5.537, 'train_steps_per_second': 0.692, 'total_flos': 4118308134912.0, 'train_loss': 0.010237421264353487, 'epoch': 3.0})

In [52]:
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.9797160243407708, 'recall': 0.9897540983606558, 'f1': 0.9847094801223242, 'number': 488}" of type <class 'dict'> for key "eval/DATE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}" of type <class 'dict'> for key "eval/LAW" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 26}" of type <class 'dict'> for key "eval/PER" as a scala

{'eval_loss': 0.013316502794623375,
 'eval_DATE': {'precision': 0.9797160243407708,
  'recall': 0.9897540983606558,
  'f1': 0.9847094801223242,
  'number': 488},
 'eval_LAW': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6},
 'eval_PER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 26},
 'eval_overall_precision': 0.9797160243407708,
 'eval_overall_recall': 0.9288461538461539,
 'eval_overall_f1': 0.9536031589338598,
 'eval_overall_accuracy': 0.9981169871794872,
 'eval_runtime': 3.9384,
 'eval_samples_per_second': 49.513,
 'eval_steps_per_second': 6.348,
 'epoch': 3.0}